# Option expiration runs on Friday after morning routine. 

In [ ]:
pwd

In [ ]:
from datetime import datetime 
import pandas as pd
import time
import numpy as np 
import os 
import blpapi
from blpapi import SubscriptionList
from IPython.display import clear_output
import pyttsx3
from win32com.client import Dispatch

engine = pyttsx3.init()
engine.setProperty('rate', 170)
engine.setProperty('voice', "HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Speech\Voices\Tokens\TTS_MS_EN-US_ZIRA_11.0")


now = datetime.now()
date= now.strftime("%m/%d/%y")
date_long = now.strftime("%m_%d_%Y")
#File name
df = pd.read_excel("option_%s.xlsx" % date_long)

In [ ]:

            
def check_last_price(df,i):
    """?"""
    if df["I/A/O"][i] =="I: (0, 0.25] Check" or df["I/A/O"][i] =="O: [-0.25,0) Check":
        if abs(df["U_U_diff"][i]) > 0.05:
            return True
        else: 
            return False
    else:
        if abs(df["U_U_diff"][i]) > 0.25:
            return True
        else: 
            return False
def allsession_lp(ticker):
    """Return the last price after market or pre-market
    ticker: the bloomberg ticker name. 
    return: the last price using all session"""


    SECURITY_DATA = blpapi.Name("securityData")
    FIELD_DATA = blpapi.Name("fieldData")
    # tickers =["FHN US EQUITY, QQQ US EQUITY"]
    # for ticker in tickers: 
    subs = SubscriptionList()
    #what does correlationID mean?
    corr=blpapi.CorrelationId(10)
    subs.add(ticker,
     "LAST_ALL_SESSIONS",
     correlationId=corr)
    session=blpapi.Session()
    options = blpapi.SessionOptions()
    options.setServerHost('localhost')
    options.setServerPort(8194)
    session = blpapi.Session(options)
    session.start()
#     print("session start!")
    session.openService("//blp/mktbar")
    session.subscribe(subs)
    ms=[]
    n =0
    close_p =0
    start = datetime.now()
    n =0
    while True:
        ev = session.nextEvent()
        n+=1
        for msg in ev:
            try:
                last_price=msg.getElement('LAST_ALL_SESSIONS')[0]
                if type(last_price) == float:
                    session.stop()
                    last_price_f = last_price
                    return last_price_f
            except:
                pass
        n+=1
        if n>=11:
            break
    session.stop()
    return last_price_f
def check_at_in(itm):
    if itm ==0:
        return "A: check"
    else: 
        #the following is the crateria to check call long 
        if abs(itm) <=1:
            if abs(itm) <=.5:
                if abs(itm) <=0.25:
                    if itm>0:
                        return "I: (0, 0.25] Check"
                    else:
                        return "O: [-0.25,0) Check"
                else:
                    if itm > 0:
                        return "I: (0.25, 0.5] check"
                    else:
                        return "O: [-0.5, -0.25) check"
            else:
                if itm>0:
                    return "I: (0.5, 1] check"
                else:
                    return "O: [-1,-0.5) check"

        else:
            if itm > 0:
                return "I : more than 1 check"
            else:
                return "O: less than -1 check"

In [ ]:


df = df.rename({'Unnamed: 0':'Name'}, axis='columns')
idx = df[df['Name']==date].index #locate the date to be today
checker1 = 0
checker2 = 0
for i in range(len(df["Name"])):
    if df["Name"][i] =="FUND":
        checker1 = i
    elif df["Name"][i] =="SHAY":
        checker2 =i
df_fund = df.iloc[checker1+2:checker2].reset_index(drop =True)
df_shay = df.iloc[checker2+2:].reset_index(drop =True)

df_shay['Ticker Name'] = [df_shay['Name'][i].split(date)[0]+"EQUITY" for i in range(len(df_shay['Name']))] # change the name
df_fund['Ticker Name'] = [df_fund['Name'][i].split(date)[0]+"EQUITY" for i in range(len(df_fund['Name']))] # change the name
df_shay['Strikes'] = [float(df_shay['Name'][i].split(date)[1][2:])for i in range(len(df_shay['Name']))]
df_shay['P/C'] = [df_shay['Name'][i].split(date)[1][1:2] for i in range(len(df_shay['Name']))]
df_fund['Strikes'] = [float(df_fund['Name'][i].split(date)[1][2:])for i in range(len(df_fund['Name']))]
df_fund['P/C'] = [df_fund['Name'][i].split(date)[1][1:2] for i in range(len(df_fund['Name']))]

df_shay["U_S_diff"] = 0
df_fund["U_S_diff"] = 0

df_shay["I/A/O"] =""
df_fund["I/A/O"] =""


shay = pd.DataFrame(columns=["Ticker","Option name","Strikes","P/C","L/S","U_S_diff","I/A/O","Quantity",
                                          "Last_I/A/O", "U_U_diff","Last_Underlying_Price","Account"])
fund = pd.DataFrame(columns=["Ticker","Option name","Strikes","P/C","L/S","U_S_diff","I/A/O","Quantity",
                                          "Last_I/A/O", "U_U_diff","Last_Underlying_Price","Account"])

shay[["Ticker","Option name","Strikes","P/C","U_S_diff","I/A/O","Quantity"]] = df_shay[["Ticker Name",'Name',"Strikes","P/C","U_S_diff","I/A/O","Quantity"]]
fund[["Ticker","Option name","Strikes","P/C","U_S_diff","I/A/O","Quantity"]] = df_fund[["Ticker Name",'Name',"Strikes","P/C","U_S_diff","I/A/O","Quantity"]]

shay["L/S"] = df_shay["Position"].apply(lambda x: "L" if x > 0 else "S")
fund["L/S"] = df_fund["Position"].apply(lambda x: "L" if x > 0 else "S")

for i in range(len(shay)):
    shay.loc[i,"Account"] = "Shay"
    shay.loc[i,"Last_I/A/O"] = ""
    shay.loc[i,"U_U_diff"] = 0
    shay.loc[i,"Last_Underlying_Price"] = 0
    fund.loc[i,"Account"] = "Fund"
    fund.loc[i,"Last_I/A/O"] = ""
    fund.loc[i,"U_U_diff"] = 0
    fund.loc[i,"Last_Underlying_Price"] = 0
    

In [ ]:
df_all = pd.concat([shay, fund], axis=0)[["Ticker","Option name","Account","Strikes","P/C","L/S","Quantity"]].reset_index(drop =True)
# drop duplicates: 
df_all_csv = df_all.drop_duplicates(subset=['Ticker',"Option name", 'Strikes',"P/C","L/S"], keep='last').reset_index(drop =True).drop(['Account'], axis=1)
df_all_csv["last"]=""
df_all_csv["current"]=""
df_all_csv["first"]=True
df_all_csv.to_csv('option_price.csv', sep='|', index=False)

In [ ]:
df_all_csv

In [ ]:

while True and datetime.now().hour <16: #close when passing 4 pm
    df_options = pd.read_csv('option_price.csv', sep='|')

    for i in range(len(df_options)):
        option_name = df_options.loc[i, 'Option name']
        ticker = df_options.loc[i, 'Ticker']
        strike = df_options.loc[i, 'Strikes']
        p_c = df_options.loc[i, 'P/C']
        l_s = df_options.loc[i, 'L/S']
        if df_options.loc[i, 'Quantity']!=0:
            try:
                last_price = allsession_lp(ticker)
                if (p_c == "P" and l_s == "L") or (p_c=="C" and l_s =="S"):
                    dif = df_options.loc[i, 'Strikes'] - last_price
                else:
                    dif = last_price - df_options.loc[i, 'Strikes']
                df_options.loc[i,"current"]= check_at_in(dif)
                if df_options.loc[i,"first"]==True:
                    df_options.loc[i,"last"] = df_options.loc[i,"current"]
                    df_options.loc[i,"first"]=False
                else:
                    print("Now check the last and the current price")
                    if df_options.loc[i,"last"] != df_options.loc[i,"current"]:
                        print("last: ",df_options.loc[i,"last"] ,"current: ",df_options.loc[i,"current"])
                        say = 'Please check option' +df_options.loc[i, 'Ticker']
                        engine.say(say)
                        engine.runAndWait()
                        if df_options.loc[i,"current"][:1] =="O":
                            send_text = "Out of money: "
                        elif df_options.loc[i,"current"][:1] =="I":
                            send_text = "In the money: "
                        elif df_options.loc[i,"current"][:1] =="A":
                            send_text = "At the money: "
                        outlook=Dispatch('Outlook.Application') #Opens outlook
                        mail = outlook.CreateItem(0) #Creates the mail item
                        mail.SentOnBehalfOfName = 'ygao@one68global.com'
                        mail.To='N.Oh@one68global.com;lluo@one68global.com'
                        mail.Subject = 'Options Expiry: {}'.format(option_name)
                        mail.HTMLBody = ("""<br><br>Underlying Stock Price: ${}
                                 <br><br>{} ${:.2f}""".format(last_price,send_text, dif))
#                         print("Sending Email!")
                        say2 = send_text + "${:.2f}".format(dif)
                        print(say2)
                        engine.say(say2)
                        engine.runAndWait()
#                         mail.Display()

                        df_options.loc[i,"last"] = df_options.loc[i,"current"]
                print(df_options.loc[i, 'Ticker'], last_price, df_options.loc[i, 'Strikes'],"Differece: ",dif)
            except Exception as e:
                print('skip %s' % ticker, e)
                continue
        else:
            print("Quantity 0!",df_options.loc[i, 'Ticker'])
    clear_output(wait=True)
    time.sleep(10)#frequency to check the strike
    df_options.to_csv('option_price.csv', sep='|', index=False)

In [ ]:
df_options